An example of how data is gathered from the NOAA API and assembled into a master dataframe that can then be easily stored in csv form.

In [5]:
import pandas as pd
import requests
import json
import time
import datetime
import numpy as np


In [8]:
headers1 = {'token': 'NOAA token goes here'}

In [ ]:
#key functions to utilize
#URL formatter function - this simply takes in a city code, and start an end dates and returns the url
#for my search of the API
def url_maker_MINMAX(city,start,end):
    url_part1 = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid='
    city_id = city
    url_part2 = '&datatypeid=TMIN,TMAX&startdate='
    dates = '{}&enddate={}'.format(start,end)
    url_part3 = '&units=standard&limit=1000'
    url = '{}{}{}{}{}'.format(url_part1,city_id,url_part2,dates,url_part3)
    return url

#pull-to-df: This function actually performs the API requests and returns a well formatted dataframe
def pull_to_df(url, header):
    trying = True
    while trying == True:
        try:
            resp = requests.get(url, headers=header, timeout=10)
            trying = False
        except:
            pass
    df = pd.DataFrame(resp.json()['results'])
    df['date'] = [df['date'][n][:-9] for n in range(0,len(df))]
    return df.pivot_table(values='value', index = 'date', columns='datatype', aggfunc=np.mean)

#finally, a date generator, which allows us to use datetime objects to find the exact date n-days ago, but
#which returns a string in the format that the NOAA API wants
def generate_date(n):
    start = datetime.datetime(2018,12,31)
    result = start - datetime.timedelta(n)
    year = str(result.year)
    month = str(result.month) if len(str(result.month))==2 else '0'+str(result.month)
    day = str(result.day) if len(str(result.day)) == 2 else '0'+str(result.day)           
    return year+'-'+month+'-'+day

Then the most important functions:

Complete pull to df: this builds on the previous functions to create a master dataframe with all the minimum and maximum temperature values for the the given city going back however many years. I cycle through numerous API keys to both expand the number of calls I can make in a single day (the limit of 10,000 calls is really quite, well, limiting) and to avoid overheating one key by making too many calls in a given minute.

create dfs: This takes in the raw data and creates a new dataframe of summary statistics, rolled up by year. Average maximum temperature that year, average min, days that breached 90. It then saves the df down to csv.

In [25]:
def complete_pull_to_df(city_id, header_list):
    df = pull_to_df(url_maker_MINMAX(city_id,generate_date(9),generate_date(0)),header_list[0])

    for n in range(1,2600):
        header = header_list[n%len(header_list)]
        url = url_maker_MINMAX(city_id,generate_date(10*n+9),generate_date(10*n))
        df = pd.concat([pull_to_df(url, header),df])
        print(f'Internal iteration: {n}')
        time.sleep(.05)
    df['dates'] = df.index
    df['dates'] = [datetime.datetime.strptime(x, '%Y-%m-%d') for x in df['dates']]
    return df

def create_dfs(raw_df,city_name):
    avg_df = pd.DataFrame()
    avg_df['dates'] = raw_df['dates']
    avg_df['TMAX'] = raw_df['TMAX']
    avg_df['TMIN'] = raw_df['TMIN']
    avg_df['years'] = [avg_df.iloc[n]['dates'].year for n in range(0,len(avg_df))]
    avg_df = avg_df.groupby('years').mean()
    
    temp_count_df = pd.DataFrame()
    temp_count_df['dates'] = raw_df['dates']
    temp_count_df['TMAX'] = raw_df['TMAX']
    temp_count_df['TMIN'] = raw_df['TMIN']
    temp_count_df['years'] = [temp_count_df.iloc[n]['dates'].year for n in range(0,len(temp_count_df))]
    temp_count_df['t90'] = [1 if temp_count_df.iloc[n]['TMAX'] > 90 else 0 for n in range(0,len(temp_count_df))]
    temp_count_df['t95'] = [1 if temp_count_df.iloc[n]['TMAX'] > 95 else 0 for n in range(0,len(temp_count_df))]
    temp_count_df['t100'] = [1 if temp_count_df.iloc[n]['TMAX'] > 100 else 0 for n in range(0,len(temp_count_df))]
    temp_count_df.drop('TMAX',axis=1,inplace=True)
    temp_count_df.drop('TMIN',axis=1,inplace=True)
    temp_count_df = temp_count_df.groupby('years').sum()
    
    comb_df = avg_df.join(temp_count_df)
    comb_df.to_csv(f'{city_name}_master.csv')
    pass

First step is to make a list of cities to go through and have them properly lined up with the city ids that NOAA has so that a city name can be referenced, but the city id can be easily pulled to generate a url for an API request.

In [9]:
#List of cities to work through:
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationid=CITY&limit=844&startdate=2018-01-01&enddate=1950-01-01&datacategoryid=TMAX&offset=470'
resp = requests.get(url, headers=headers1)


In [10]:
cities = pd.DataFrame(resp.json()['results'])
cities.head()

,datacoverage,id,maxdate,mindate,name
0,1.0,CITY:US000001,2019-08-17,1870-11-01,"Washington D.C., US"
1,1.0,CITY:US010001,2019-08-17,1895-11-01,"Alexander City, AL US"
2,1.0,CITY:US010002,2019-08-17,1903-02-01,"Anniston, AL US"
3,1.0,CITY:US010003,2019-08-17,1906-04-01,"Auburn, AL US"
4,1.0,CITY:US010004,2019-08-17,1900-10-01,"Birmingham, AL US"


In [11]:
cities['name'] = [x[:-3] for x in cities['name']]
cities['name'][0] = cities['name'][0][:-1]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
cities.set_index('name', inplace=True)
cities.head()

,datacoverage,id,maxdate,mindate
name,,,,
Washington D.C.,1.0,CITY:US000001,2019-08-17,1870-11-01
"Alexander City, AL",1.0,CITY:US010001,2019-08-17,1895-11-01
"Anniston, AL",1.0,CITY:US010002,2019-08-17,1903-02-01
"Auburn, AL",1.0,CITY:US010003,2019-08-17,1906-04-01
"Birmingham, AL",1.0,CITY:US010004,2019-08-17,1900-10-01


In [13]:
#I wanted to use the 100 most populous cities as per wikipedia, what follows is that list, 
#written as they exist in the NOAA pull, minus a couple of exceptions
#Aurora CO not in the NOAA list, perhaps too close to Denver
#Ditto Henderson NV and Las Vegas, Chula Vista and San Diego, Irvine and santa ana,
#Chandler AZ and mesa
city_names = ['Washington D.C.','New York, NY','Los Angeles, CA','Chicago, IL','Houston, TX', 'Phoenix, AZ',
             'Philadelphia, PA', 'San Antonio, TX','San Diego, CA','Dallas, TX','San Jose, CA','Austin, TX',
             'Jacksonville, FL','Fort Worth, TX','Columbus, OH', 'San Francisco, CA','Charlotte, NC',
             'Indianapolis, IN', 'Seattle, WA','Denver, CO','Boston, MA','El Paso, TX','Detroit, MI',
             'Nashville, TN','Portland, OR','Memphis, TN','Oklahoma City, OK','Las Vegas, NV','Louisville, KY',
             'Baltimore, MD','Milwaukee, WI','Albuquerque, NM','Tucson, AZ','Fresno, CA','Mesa, AZ',
             'Sacramento, CA','Atlanta, GA','Kansas City, MO','Colorado Springs, CO','Miami, FL', 'Raleigh, NC',
             'Omaha, NE','Long Beach, CA','Virginia Beach, VA','Oakland, CA','Minneapolis, MN','Tulsa, OK',
             'Arlington, TX','Tampa, FL', 'New Orleans, LA','Wichita, KS','Cleveland, OH','Bakersfield, CA',
             'Honolulu, HI','Santa Ana, CA','Riverside, CA','Corpus Christi, TX','Lexington, KY','Stockton, CA',
             'Saint Paul, MN','St. Louis, MO','Cincinnati, OH','Pittsburgh, PA','Greensboro, NC','Anchorage, AK',
             'Plano, TX','Lincoln, NE','Orlando, FL','Newark, NJ','Toledo, OH','Durham, NC','Fort Wayne, IN',
             'Jersey City, NJ','St. Petersburg, FL','Laredo, TX','Madison, WI','Buffalo, NY','Lubbock, TX',
             'Winston-Salem, NC','Norfolk, VA','Chesapeake, VA','Boise, ID','Richmond, VA',
             'Baton Rouge, LA','Spokane, WA','Des Moines, IA','Tacoma, WA']

In [114]:
cities.loc[city_names]

,datacoverage,id,maxdate,mindate
name,,,,
Washington D.C.,1.0,CITY:US000001,2019-08-04,1870-11-01
"New York, NY",1.0,CITY:US360019,2019-08-04,1869-01-01
"Los Angeles, CA",1.0,CITY:US060013,2019-08-04,1893-01-01
"Chicago, IL",1.0,CITY:US170006,2019-08-04,1870-10-15
"Houston, TX",1.0,CITY:US480031,2019-08-04,1883-04-01
"Phoenix, AZ",1.0,CITY:US040011,2019-08-04,1893-01-01
"Philadelphia, PA",1.0,CITY:US420015,2019-08-04,1893-01-01
"San Antonio, TX",1.0,CITY:US480057,2019-08-04,1893-01-01
"San Diego, CA",1.0,CITY:US060030,2019-08-04,1850-10-01


In [14]:
headers = ['A list of tokens might go here']
for n in range(0,len(headers)*3): #how many cities you can do in a day limited by the number of tokens you have
    city = city_names[n]
    city_id = cities.loc[city]['id']

    raw_df = complete_pull_to_df(city_id, headers)
    create_dfs(raw_df,city)
    print(f'n: {n} -- city: {city}')